In [2]:
!pip install pysus requests

In [127]:
import pandas as pd

df_painel_geral = pd.DataFrame()

# IBGE

In [3]:
import requests
import pprint

link = "https://servicodados.ibge.gov.br/api/v3/agregados/6579/periodos/2014|2015|2016|2017|2018|2019|2020|2021|2022|2023|2024|2025/variaveis/9324?localidades=N1[all]|N3[53]"


requisicao = requests.get(link)
json_infos = requisicao.json()

estimativa_popDF = json_infos[0]['resultados'][0]['series'][1]['serie']

for(chave, valor) in estimativa_popDF.items():
  print(chave, valor)

# pprint.pprint(json_infos)






2014 2852372
2015 2914830
2016 2977216
2017 3039444
2018 2974703
2019 3015268
2020 3055149
2021 3094325
2024 2982818
2025 2996899


# Taxa de homicídios no DF - SIM


In [5]:
from pysus import SIM
import pandas as pd
sim = SIM().load() # Loads the files from DATASUS
years = [i for i in range(2014,2026)]
#
sim_files = sim.get_files("CID10", uf=["DF"], year=years)
#
parquets = sim.download(sim_files)

DODF2024.parquet: 100%|██████████| 38.9k/38.9k [00:02<00:00, 19.0kB/s]


## Salvar dados Brutos SIM

In [23]:
for parquet in parquets:
  df = pd.read_parquet(str(parquet))
  print()
  df.to_parquet('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_brutos/SIM/'+str(parquet).split('/')[-1], engine='pyarrow', compression='snappy')

In [28]:
padrao = "^(X8[5-9]|X9[0-9]|Y0[0-9]|Y3[5-6])"
for parquet, (chave, valor) in zip(parquets, estimativa_popDF.items()):
  try:
    df = pd.DataFrame(parquet.to_dataframe())

    filtro_causabas = df.CAUSABAS.str.contains(padrao, na=False)
    print(len(df[filtro_causabas]))
    print("Ano: " + chave)
    print("Valor: " + valor)
    print("Taxa: " + str((int(len(df[filtro_causabas]))*100000/int(valor))))
    print("Quantidade de mortos: " + str(len(df.CAUSABAS)))
  except ValueError:
        print(f"\nAviso: O arquivo do ano {chave} está vazio ou corrompido. Pulando...\n")
  except Exception as e:
        print(f"\nErro inesperado no ano {chave}: {e}\n")


/tmp/ipython-input-2428162509.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro_causabas = df.CAUSABAS.str.contains(padrao, na=False)


843
Ano: 2014
Valor: 2852372
Taxa: 29.554349853385183
Quantidade de mortos: 12018


/tmp/ipython-input-2428162509.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro_causabas = df.CAUSABAS.str.contains(padrao, na=False)


742
Ano: 2015
Valor: 2914830
Taxa: 25.456030025764797
Quantidade de mortos: 11975


/tmp/ipython-input-2428162509.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro_causabas = df.CAUSABAS.str.contains(padrao, na=False)


...

/tmp/ipython-input-2428162509.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro_causabas = df.CAUSABAS.str.contains(padrao, na=False)


401
Ano: 2021
Valor: 3094325
Taxa: 12.959207581621193
Quantidade de mortos: 19079


/tmp/ipython-input-2428162509.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro_causabas = df.CAUSABAS.str.contains(padrao, na=False)


357
Ano: 2024
Valor: 2982818
Taxa: 11.968547863128089
Quantidade de mortos: 14457
347
Ano: 2025
Valor: 2996899
Taxa: 11.578635115831398
Quantidade de mortos: 14079


/tmp/ipython-input-2428162509.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro_causabas = df.CAUSABAS.str.contains(padrao, na=False)


## Detalhamento

In [25]:
def calcular_idade(valor):
    # Transforma em string para garantir que o startswith funcione
    valor_str = str(valor)
    if valor_str.startswith('4'):
        # Pega do segundo caractere em diante e converte para inteiro
        return int(valor_str[1:])
    elif valor_str.startswith('5'):
        return 100 + int(valor_str[1:])
    return 0

# def codigo_ibge(valor):
#     # Transforma em string e remove espaços em branco
#     valor_str = str(valor).strip()

#     prefixo = int(valor_str[:2])

#     ufs_validas =  [
#         11, 12, 13, 14, 15, 16, 17, # Norte
#         21, 22, 23, 24, 25, 26, 27, 28, 29, # Nordeste
#         31, 32, 33, 35, # Sudeste
#         41, 42, 43, # Sul
#         50, 51, 52, 53 # Centro-Oeste
#     ]

#     if prefixo in ufs_validas:
#         return prefixo

#     return '0' # Retorna string '0' para manter o padrão de tipo (texto)


dfs_lista_sim = []
for parquet in parquets:
  df = pd.DataFrame(parquet.to_dataframe())
  dfs_lista_sim.append(df)





In [26]:
dfs_lista_sim[3].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12514 entries, 0 to 12513
Data columns (total 88 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CONTADOR    12514 non-null  string
 1   ORIGEM      12514 non-null  string
 2   TIPOBITO    12514 non-null  string
 3   DTOBITO     12514 non-null  string
 4   HORAOBITO   12514 non-null  string
 5   NATURAL     12514 non-null  string
 6   CODMUNNATU  12514 non-null  string
 7   DTNASC      12514 non-null  string
 8   IDADE       12514 non-null  string
 9   SEXO        12514 non-null  Int64 
 10  RACACOR     12514 non-null  string
 11  ESTCIV      12514 non-null  string
 12  ESC         12514 non-null  string
 13  ESC2010     12514 non-null  string
 14  SERIESCFAL  12514 non-null  string
 15  OCUP        12514 non-null  string
 16  CODMUNRES   12514 non-null  Int64 
 17  LOCOCOR     12514 non-null  string
 18  CODESTAB    12514 non-null  string
 19  ESTABDESCR  12514 non-null  string
 20  CODMUN

## Taxas de Homicídios Detalhados

In [128]:
# for (chave, valor) in df.isnull().sum().items():
#   print(f"chave: {chave}, valor: {valor}")

df_homicidios = pd.DataFrame()
for df, (ano, quantidade) in zip(dfs_lista_sim, estimativa_popDF.items()):
  try:
    df['idade_obito'] = df['IDADE'].apply(calcular_idade)
    # df['uf_ibge'] = df['CODMUNOCOR'].apply(codigo_ibge)
    df['ESC'] = pd.to_numeric(df['ESC'], errors='coerce').fillna(0).astype(int)
    df.fillna('0')

    #filtros
    filtro_causabas = df.CAUSABAS.str.contains(padrao, na=False)
    df_cid_10 = df[filtro_causabas]

    filtro_idade_jovens = (df_cid_10['idade_obito'] >= 15) & (df_cid_10['idade_obito'] <= 29)
    filtro_sexo_masculino = df_cid_10['SEXO'] == 1

    df_cid_10['RACACOR'] = df_cid_10['RACACOR'].astype(str)
    filtro_negro_pardo = df_cid_10['RACACOR'].isin(['2', '4'])

    norte = [11, 12, 13, 14, 15, 16, 17]
    nordeste = [21, 22, 23, 24, 25, 26, 27, 28, 29]
    # filtro_norte_nordeste = df['uf_ibge'].isin(norte + nordeste)

    padrao = "^(X8[5-9]|X9[0-9]|Y0[0-9]|Y3[5-6])"
    #Escolaridade baixa: menor que 12
    filtro_esc_baixa = df_cid_10['ESC'] <= 4


    quantidade_pardos_negros_under_2017 = df_cid_10.groupby(['RACACOR'])['RACACOR'].count()[2] + df_cid_10.groupby(['RACACOR'])['RACACOR'].count()[4]
    quantidade_homicidios = len(df_cid_10)
    quantidade_homicidios_jovens = len(df_cid_10[filtro_idade_jovens])
    quantidade_homicidios_masculino = len(df_cid_10[filtro_sexo_masculino])
    quantidade_homicidios_negro_pardo = df_cid_10[df_cid_10['RACACOR'] == '2'].groupby(['RACACOR'])['RACACOR'].count().sum() + df_cid_10[df_cid_10['RACACOR'] == '4'].groupby(['RACACOR'])['RACACOR'].count().sum() if int(ano) > 2017 else quantidade_pardos_negros_under_2017
    quantidade_pessoas_escolaridade_baixa = len(df_cid_10[filtro_esc_baixa])
    # quantidade_homicidios_norte_nordeste = len(df[filtro_norte_nordeste])

    quantidade = int(quantidade)

    taxa_homicidios = round(quantidade_homicidios*100000/quantidade, 2)
    taxa_homicidios_jovens = round(quantidade_homicidios_jovens*100000/quantidade,2)
    taxa_homicidios_masculino = round(quantidade_homicidios_masculino*100000/quantidade,2)
    taxa_homicidios_negro_pardo = round(quantidade_homicidios_negro_pardo*100000/quantidade,2)
    taxa_homicidios_baixa_escolaridade = round(quantidade_pessoas_escolaridade_baixa*100000/quantidade,2)

    print(f"Ano: {ano}")
    print(f"Quantidade de homicídios: {quantidade_homicidios}, Taxa: {quantidade_homicidios*100000/quantidade}")
    print("-----------------------------------------------------------------")
    print(f"Quantidade de homicídios jovens: {quantidade_homicidios_jovens}, Taxa: {taxa_homicidios}")
    print(f"Quantidade de homicídios de homens: {quantidade_homicidios_masculino}, Taxa: {taxa_homicidios_masculino}")
    print(f"Quantidade de homicídios de pessoas negras e pardas: {quantidade_homicidios_negro_pardo}, Taxa: {taxa_homicidios_negro_pardo}")
    # print(f"Quantidade de homicídios no norte e nordeste: {quantidade_homicidios_norte_nordeste}, Taxa: {quantidade_homicidios_norte_nordeste*100000/quantidade}")
    print(f"Quantidade de pessoas com escolaridade baixa que foram mortas: {quantidade_pessoas_escolaridade_baixa}, Taxa: {taxa_homicidios_baixa_escolaridade}")
    print("\n\n")
    df_novo = pd.DataFrame({'SIM Ano':[ano],
                            'Homicidios':[quantidade_homicidios],'Taxa Homicídios': [taxa_homicidios],
                            'Jovens':[quantidade_homicidios_jovens],'Taxa Jovens': [taxa_homicidios_jovens],
                            'Homens':[quantidade_homicidios_masculino],'Taxa Homens': [taxa_homicidios_masculino],
                            'Negro e Pardos': [quantidade_homicidios_negro_pardo],'Taxa Negros e Pardos': [taxa_homicidios_negro_pardo],
                            'Escolaridade Baixa': [quantidade_pessoas_escolaridade_baixa], 'Taxa Escolaridade Baixa': [taxa_homicidios_baixa_escolaridade]
                            })
    racas = df_cid_10['RACACOR'].value_counts()
    print(racas)

    df_homicidios = pd.concat([df_homicidios, df_novo], ignore_index=True)
  except ValueError:
        print(f"\nAviso: O arquivo do ano {ano} está vazio ou corrompido. Pulando...\n")
  except Exception as e:
        print(f"\nErro inesperado no ano {ano}: {e}\n")






/tmp/ipython-input-3130589066.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro_causabas = df.CAUSABAS.str.contains(padrao, na=False)
/tmp/ipython-input-3130589066.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cid_10['RACACOR'] = df_cid_10['RACACOR'].astype(str)
/tmp/ipython-input-3130589066.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quantidade_pardos_negros_under_2017 = df_cid_10.groupby(['RACACOR'])['RACACOR'].count()[2] + df_cid_10.groupby(['

Ano: 2014
Quantidade de homicídios: 843, Taxa: 29.554349853385183
-----------------------------------------------------------------
Quantidade de homicídios jovens: 453, Taxa: 29.55
Quantidade de homicídios de homens: 783, Taxa: 27.45
Quantidade de homicídios de pessoas negras e pardas: 710, Taxa: 24.89
Quantidade de pessoas com escolaridade baixa que foram mortas: 782, Taxa: 27.42



RACACOR
4          652
1          117
2           58
            15
3            1
Name: count, dtype: int64
Ano: 2015
Quantidade de homicídios: 742, Taxa: 25.456030025764797
-----------------------------------------------------------------
Quantidade de homicídios jovens: 382, Taxa: 25.46
Quantidade de homicídios de homens: 682, Taxa: 23.4
Quantidade de homicídios de pessoas negras e pardas: 598, Taxa: 20.52
Quantidade de pessoas com escolaridade baixa que foram mortas: 680, Taxa: 23.33



RACACOR
4          546
1          131
2           52
            11
3            2
Name: count, dtype: int64


/tmp/ipython-input-3130589066.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro_causabas = df.CAUSABAS.str.contains(padrao, na=False)
/tmp/ipython-input-3130589066.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cid_10['RACACOR'] = df_cid_10['RACACOR'].astype(str)
/tmp/ipython-input-3130589066.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quantidade_pardos_negros_under_2017 = df_cid_10.groupby(['RACACOR'])['RACACOR'].count()[2] + df_cid_10.groupby(['


Erro inesperado no ano 2016: index 4 is out of bounds for axis 0 with size 4

Ano: 2017
Quantidade de homicídios: 610, Taxa: 20.069460072302697
-----------------------------------------------------------------
Quantidade de homicídios jovens: 325, Taxa: 20.07
Quantidade de homicídios de homens: 564, Taxa: 18.56
Quantidade de homicídios de pessoas negras e pardas: 486, Taxa: 15.99
Quantidade de pessoas com escolaridade baixa que foram mortas: 575, Taxa: 18.92



RACACOR
4          445
1          116
2           41
3            4
             3
5            1
Name: count, dtype: int64


/tmp/ipython-input-3130589066.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro_causabas = df.CAUSABAS.str.contains(padrao, na=False)
/tmp/ipython-input-3130589066.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cid_10['RACACOR'] = df_cid_10['RACACOR'].astype(str)
/tmp/ipython-input-3130589066.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quantidade_pardos_negros_under_2017 = df_cid_10.groupby(['RACACOR'])['RACACOR'].count()[2] + df_cid_10.groupby(['

...

Ano: 2020
Quantidade de homicídios: 453, Taxa: 14.827427402067787
-----------------------------------------------------------------
Quantidade de homicídios jovens: 219, Taxa: 14.83
Quantidade de homicídios de homens: 418, Taxa: 13.68
Quantidade de homicídios de pessoas negras e pardas: 326, Taxa: 10.67
Quantidade de pessoas com escolaridade baixa que foram mortas: 397, Taxa: 12.99



RACACOR
4    281
1    117
2     45
3      5
       5
Name: count, dtype: int64


/tmp/ipython-input-3130589066.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtro_causabas = df.CAUSABAS.str.contains(padrao, na=False)
/tmp/ipython-input-3130589066.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cid_10['RACACOR'] = df_cid_10['RACACOR'].astype(str)
/tmp/ipython-input-3130589066.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quantidade_pardos_negros_under_2017 = df_cid_10.groupby(['RACACOR'])['RACACOR'].count()[2] + df_cid_10.groupby(['

Ano: 2021
Quantidade de homicídios: 401, Taxa: 12.959207581621193
-----------------------------------------------------------------
Quantidade de homicídios jovens: 168, Taxa: 12.96
Quantidade de homicídios de homens: 355, Taxa: 11.47
Quantidade de homicídios de pessoas negras e pardas: 304, Taxa: 9.82
Quantidade de pessoas com escolaridade baixa que foram mortas: 345, Taxa: 11.15



RACACOR
4    265
1     85
2     39
      10
3      2
Name: count, dtype: int64
Ano: 2024
Quantidade de homicídios: 357, Taxa: 11.968547863128089
-----------------------------------------------------------------
Quantidade de homicídios jovens: 153, Taxa: 11.97
Quantidade de homicídios de homens: 321, Taxa: 10.76
Quantidade de homicídios de pessoas negras e pardas: 269, Taxa: 9.02
Quantidade de pessoas com escolaridade baixa que foram mortas: 308, Taxa: 10.33



RACACOR
4    233
1     68
2     36
      17
5      2
3      1
Name: count, dtype: int64


/tmp/ipython-input-3130589066.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cid_10['RACACOR'] = df_cid_10['RACACOR'].astype(str)
/tmp/ipython-input-3130589066.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  quantidade_pardos_negros_under_2017 = df_cid_10.groupby(['RACACOR'])['RACACOR'].count()[2] + df_cid_10.groupby(['RACACOR'])['RACACOR'].count()[4]
/tmp/ipython-input-3130589066.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with Dat

Ano: 2025
Quantidade de homicídios: 347, Taxa: 11.578635115831398
-----------------------------------------------------------------
Quantidade de homicídios jovens: 136, Taxa: 11.58
Quantidade de homicídios de homens: 302, Taxa: 10.08
Quantidade de homicídios de pessoas negras e pardas: 264, Taxa: 8.81
Quantidade de pessoas com escolaridade baixa que foram mortas: 303, Taxa: 10.11



RACACOR
4    222
1     71
2     42
      10
5      2
Name: count, dtype: int64


In [129]:
df_homicidios.to_excel('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_filtrados/SIM/homicidios_df.xlsx', index=False)

## Adicionando ao painel

In [130]:
df_painel_geral = pd.concat([df_painel_geral, df_homicidios], ignore_index=True)

# Taxa de Roubos - SINESP

In [48]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

sinesp_dados_brutos = pd.read_excel('/content/drive/MyDrive/criminalidade_df/data/raw/SINESP/sinesp_dados_brutos.xlsx')
sinesp_dados_brutos.to_excel('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_brutos/SINESP/sinesp_dados_brutos.xlsx', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
print(sinesp_dados_brutos.isna().sum())
roubos = ['Furto de veículo', 'Roubo a instituição financeira', 'Roubo de carga', 'Roubo de veículo', 'Roubo seguido de morte (latrocínio)']

# sinesp_df = sinesp_dados_brutos[(sinesp_dados_brutos['UF'] == 'Distrito Federal')]
sinesp_df = sinesp_dados_brutos[(sinesp_dados_brutos['UF'] == 'Distrito Federal') & (sinesp_dados_brutos['Tipo Crime'].isin(roubos))]

sinesp_df.groupby(['Tipo Crime'])['Ocorrências'].sum()

UF             0
Tipo Crime     0
Ano            0
Mês            0
Ocorrências    0
dtype: int64


,Ocorrências
Tipo Crime,
Furto de veículo,40589
Roubo a instituição financeira,21
Roubo de carga,331
Roubo de veículo,28496
Roubo seguido de morte (latrocínio),243


## Detalhamento Taxas de Roubos e Furtos

In [131]:

df_furtos_roubos = pd.DataFrame()
for ano, (chave, valor) in zip(range(2014, 2026), estimativa_popDF.items()):

  #quantidade total
  quantidade = sinesp_df[sinesp_df['Ano'] == ano].groupby('Tipo Crime')['Ocorrências'].sum().sum()
  #furto
  quantidade_furto = sinesp_df[(sinesp_df['Ano'] == ano) & (sinesp_df['Tipo Crime'] == 'Furto de veículo')].groupby(['Tipo Crime'])['Ocorrências'].sum().sum()

  #roubo
  quantidade_roubo_instituicao = sinesp_df[(sinesp_df['Ano'] == ano) & (sinesp_df['Tipo Crime'] == 'Roubo a instituição financeira')].groupby(['Tipo Crime'])['Ocorrências'].sum().sum()
  quantidade_roubo_carga = sinesp_df[(sinesp_df['Ano'] == ano) & (sinesp_df['Tipo Crime'] == 'Roubo de carga')].groupby(['Tipo Crime'])['Ocorrências'].sum().sum()
  quantidade_roubo_veiculo = sinesp_df[(sinesp_df['Ano'] == ano) & (sinesp_df['Tipo Crime'] == 'Roubo de veículo')].groupby(['Tipo Crime'])['Ocorrências'].sum().sum()
  quantidade_roubo_sg_morte = sinesp_df[(sinesp_df['Ano'] == ano) & (sinesp_df['Tipo Crime'] == 'Roubo seguido de morte (latrocínio)')].groupby(['Tipo Crime'])['Ocorrências'].sum().sum()

  #taxas
  taxa_furto_veiculos = round(100000*quantidade_furto/int(valor),2)
  taxa_roubo_carga = round(100000*quantidade_roubo_carga/int(valor),2)
  taxa_roubo_instituicao = round(100000*quantidade_roubo_instituicao/int(valor),2)
  taxa_roubo_sg_morte = round(100000*quantidade_roubo_sg_morte/int(valor),2)
  taxa_roubo_veiculo = round(100000*quantidade_roubo_veiculo/int(valor),2)
  taxa_total = round(100000*quantidade/int(valor),2)


  print('Ano: ' + str(ano))

  print(f"Quantidade de furtos de veículos: {quantidade_furto}, Taxa: {taxa_furto_veiculos}")
  print(f"Quantidade de roubos de carga: {quantidade_roubo_carga}, Taxa: {taxa_roubo_carga}")
  print(f"Quantidade de roubos de instituição financeira: {quantidade_roubo_instituicao}, Taxa: {taxa_roubo_instituicao}")
  print(f"Quantidade de roubos seguidos de mote: {quantidade_roubo_sg_morte}, Taxa: {taxa_roubo_sg_morte}")
  print(f"Quantidade de roubos de veículos: {quantidade_roubo_veiculo}, Taxa: {taxa_roubo_veiculo}")

  print("Total: " + str(quantidade))
  print(f"Taxa: {str(100000*quantidade/int(valor))} \n Ano Referência IBGE: {chave}")
  # print(sinesp_df[sinesp_df['Ano'] == ano].groupby(['Tipo Crime'])['Ocorrências'].sum())
  print('\n\n')
  df_novo = pd.DataFrame({'Sinesp Ano':[ano],
                          'Furto de Veículos':[quantidade_furto],'Taxa Furto de Veículos': [taxa_furto_veiculos],
                          'Roubo de Carga':[quantidade_roubo_carga],'Taxa Roubo de Carga': [taxa_roubo_carga],
                          'Roubo de Instituição Financeira':[quantidade_roubo_instituicao], 'Taxa Roubo de Instituição Financeira': [taxa_roubo_instituicao],
                          'Roubo de Veículo':[quantidade_roubo_veiculo],'Taxa Roubo de Veículo': [taxa_roubo_veiculo],
                          'Roubo Seguido de Morte':[quantidade_roubo_sg_morte],'Taxa Roubo Seguido de Morte': [taxa_roubo_sg_morte],
                          'Taxa Total': [taxa_total], 'Total': [quantidade]})
  df_furtos_roubos = pd.concat([df_furtos_roubos, df_novo], ignore_index=True)


print(sinesp_df.groupby(['Tipo Crime'])['Ocorrências'].sum())
print("Total: " + str(sinesp_df.groupby('Tipo Crime')['Ocorrências'].sum().sum()))


Ano: 2014
Quantidade de furtos de veículos: 0, Taxa: 0.0
Quantidade de roubos de carga: 0, Taxa: 0.0
Quantidade de roubos de instituição financeira: 0, Taxa: 0.0
Quantidade de roubos seguidos de mote: 0, Taxa: 0.0
Quantidade de roubos de veículos: 0, Taxa: 0.0
Total: 0
Taxa: 0.0 
 Ano Referência IBGE: 2014



Ano: 2015
Quantidade de furtos de veículos: 6500, Taxa: 223.0
Quantidade de roubos de carga: 31, Taxa: 1.06
Quantidade de roubos de instituição financeira: 8, Taxa: 0.27
Quantidade de roubos seguidos de mote: 45, Taxa: 1.54
Quantidade de roubos de veículos: 4807, Taxa: 164.92
Total: 11391
Taxa: 390.79466040901184 
 Ano Referência IBGE: 2015



Ano: 2016
Quantidade de furtos de veículos: 6924, Taxa: 232.57
Quantidade de roubos de carga: 67, Taxa: 2.25
Quantidade de roubos de instituição financeira: 5, Taxa: 0.17
Quantidade de roubos seguidos de mote: 42, Taxa: 1.41
Quantidade de roubos de veículos: 5624, Taxa: 188.9
Total: 12662
Taxa: 425.2966529805026 
 Ano Referência IBGE: 2016



In [132]:
df_furtos_roubos.to_excel('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_filtrados/SINESP/furtos_roubos_df.xlsx', index=False)

## Adicionando ao painel geral

In [133]:
df_painel_geral = pd.concat([df_painel_geral, df_furtos_roubos], ignore_index=True)

# Internações hospitalares por agressão (CID X85–Y09) e por transtornos mentais (CID F00–F99) - SIH

In [90]:
from pysus import SIH
sih = SIH().load()

In [91]:
years = [i for i in range(2014,2026)]
sih_files = sih.get_files("RD", uf="DF", year=years)
sih_parquets = sih.download(sih_files)


RDDF2512.parquet: 100%|██████████| 71.5k/71.5k [00:03<00:00, 20.1kB/s]


In [111]:
for parquet in sih_parquets:
  df = pd.read_parquet(str(parquet))
  print()
  df.to_parquet('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_brutos/SIH/'+str(parquet).split('/')[-1], engine='pyarrow', compression='snappy')

In [112]:
import pandas as pd
colunas_interesse = ['ANO_CMPT', 'SEXO', 'DIAG_PRINC']
try:
  sih_df_concat = pd.read_parquet('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_processados/sih_concat.parquet', columns=colunas_interesse)
  print('Arquivo parquet carregado!')
except FileNotFoundError:
  sih_df_concat = pd.DataFrame()


  frames = []
  for i in sih_parquets:
    df_temporario = pd.read_parquet(str(i), columns=colunas_interesse)
    frames.append(df_temporario)

  sih_df_concat = pd.concat(frames, ignore_index=True)
# sih_df_concat.fillna(-1)


Arquivo parquet carregado!


In [113]:
print(sih_df_concat.isna().sum().sum())

0


In [114]:
sih_df_concat.head()

,ANO_CMPT,SEXO,DIAG_PRINC
0,2014,3,I269
1,2014,1,I500
2,2014,3,I64
3,2014,3,E138
4,2014,3,B209


In [115]:
sih_df_concat.to_parquet('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_processados/sih_concat.parquet', compression='snappy')

In [116]:
sih_df_concat['SEXO'] = sih_df_concat['SEXO'].astype(int)

padrao_agressao = "^(X8[5-9]|X9[0-9]|Y0[0-9])"
padrao_transt = "^(F[0-9][0-9])"
sih_df_agre = sih_df_concat.DIAG_PRINC.str.contains(padrao_agressao, na=False)
sih_df_transt = sih_df_concat.DIAG_PRINC.str.contains(padrao_transt, na=False)

/tmp/ipython-input-3238002102.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sih_df_agre = sih_df_concat.DIAG_PRINC.str.contains(padrao_agressao, na=False)
/tmp/ipython-input-3238002102.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sih_df_transt = sih_df_concat.DIAG_PRINC.str.contains(padrao_transt, na=False)


In [117]:
sih_df_concat.SEXO.value_counts()

,count
SEXO,
3,1541994
1,1022765


In [134]:
df_agressao_transt = pd.DataFrame()
for (ano, valor) in estimativa_popDF.items():


  quantidade_agressao = len(sih_df_concat[sih_df_agre & (sih_df_concat.ANO_CMPT == ano)])
  quantidade_agressao_homem = len(sih_df_concat[sih_df_agre & (sih_df_concat.ANO_CMPT == ano) & (sih_df_concat['SEXO'] == 1)])
  quantidade_agressao_mulher = len(sih_df_concat[sih_df_agre & (sih_df_concat.ANO_CMPT == ano) & (sih_df_concat['SEXO'] == 3)])
  quantidade_transt = len(sih_df_concat[sih_df_transt & (sih_df_concat.ANO_CMPT == ano)])
  quantidade_transt_mulher = len(sih_df_concat[sih_df_transt & (sih_df_concat.ANO_CMPT == ano) & (sih_df_concat['SEXO'] == 3)])
  quantidade_transt_homem = len(sih_df_concat[sih_df_transt & (sih_df_concat.ANO_CMPT == ano) & (sih_df_concat['SEXO'] == 1)])

  total = len(sih_df_concat[(sih_df_concat.ANO_CMPT == ano) & (sih_df_agre | sih_df_transt)])

  print(f"Ano: {ano}")
  print(f"Transtonos Mentais: {quantidade_transt}")
  print(f"Violência Agressões: {quantidade_agressao}")
  print(f"quantidade_agressao_homem {quantidade_agressao_homem}")
  print(f"quantidade_agressao_mulher {quantidade_agressao_mulher}")
  print(f"quantidade trauma mulher {quantidade_transt_mulher}")
  print(f"Total: {total}")

  df_novo = pd.DataFrame({'SIH Ano':[ano],
                          'Total':total,
                          'Transt':[quantidade_transt],
                          'Transt Homens':[quantidade_transt_homem],
                          'Transt Mulheres':[quantidade_transt_mulher],
                          'Agressao':[quantidade_agressao],
                          'Agressao Homens':[quantidade_agressao_homem],
                          'Agressao Mulheres':[quantidade_agressao_mulher]
                          })
  df_agressao_transt = pd.concat([df_agressao_transt, df_novo], ignore_index=True)




Ano: 2014
Transtonos Mentais: 3758
Violência Agressões: 21
quantidade_agressao_homem 19
quantidade_agressao_mulher 2
quantidade trauma mulher 1693
Total: 3779
Ano: 2015
Transtonos Mentais: 4090
Violência Agressões: 17
quantidade_agressao_homem 14
quantidade_agressao_mulher 3
quantidade trauma mulher 1832
Total: 4107
Ano: 2016
Transtonos Mentais: 4056
Violência Agressões: 0
quantidade_agressao_homem 0
quantidade_agressao_mulher 0
quantidade trauma mulher 1906
Total: 4056
Ano: 2017
Transtonos Mentais: 4244
Violência Agressões: 0
quantidade_agressao_homem 0
quantidade_agressao_mulher 0
quantidade trauma mulher 2090
Total: 4244
Ano: 2018
Transtonos Mentais: 5118
Violência Agressões: 0
quantidade_agressao_homem 0
quantidade_agressao_mulher 0
quantidade trauma mulher 2549
Total: 5118
Ano: 2019
Transtonos Mentais: 6021
Violência Agressões: 0
quantidade_agressao_homem 0
quantidade_agressao_mulher 0
quantidade trauma mulher 3107
Total: 6021
Ano: 2020
Transtonos Mentais: 4821
Violência Agressões

In [135]:
df_agressao_transt.to_excel('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_filtrados/SIH/agressao_transt_df.xlsx', index=False)

## Adicionando ao painel geral

In [136]:
df_painel_geral = pd.concat([df_painel_geral, df_agressao_transt], ignore_index=True)

# Taxa de estupros e violência doméstica - SINAN

In [53]:
from pysus import SINAN
import pandas as pd

sinan = SINAN().load() # Loads the files from DATASUS


years = [i for i in range(2014,2026)]

sinan_files = sinan.get_files(dis_code=["VIOL"], year=years)
sinan_parquets = sinan.download(sinan_files)

VIOLBR24.parquet: 100%|██████████| 1.90M/1.90M [02:34<00:00, 12.3kB/s]


In [54]:
for parquet in sinan_parquets:
  df = pd.read_parquet(str(parquet))
  print()
  df.to_parquet('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_brutos/SINAN/'+str(parquet).split('/')[-1], engine='pyarrow', compression='snappy')

In [71]:
# sinan_parquets[0].to_dataframe().columns.to_list()
sinan_parquets[0].to_dataframe()['CS_ESCOL_N'].unique()

<StringArray>
['09', '05', '08', '04', '03', '10', '06', '00', '02', '01', '07']
Length: 11, dtype: string

In [83]:
def taxa(quantidade, populacao):
  return float(100000*int(quantidade)/int(populacao))

In [137]:
df_estupro_viol = pd.DataFrame()
for df_parquet, ano, (chave, valor) in zip(sinan_parquets, range(2014, 2026), estimativa_popDF.items()):
  df = pd.DataFrame(df_parquet.to_dataframe())

  # df_sinan = df[(df['ID_MUNICIP'] == '530010') & df.ID_AGRAVO.str.contains(padrao, na=False)] Já vem com filtro de VIOL
  df_sinan = df[(df['ID_MUNICIP'] == '530010')]

  df_sinan['CS_RACA'] = pd.to_numeric(df_sinan['CS_RACA'], errors='coerce').fillna(0).astype(int)
  df_sinan['CS_ESCOL_N'] = pd.to_numeric(df_sinan['CS_RACA'], errors='coerce').fillna(0).astype(int)
  # filtros
  filtro_homem = df_sinan['CS_SEXO'] == 'M'
  filtro_mulher = df_sinan['CS_SEXO'] == 'F'
  filtro_violencia_sexual = df_sinan['VIOL_SEXU'] == '1'
  filtro_viol_domestica = df_sinan['LOCAL_OCOR'] == '01'
  filtro_negro_pardo = df_sinan['CS_RACA'].isin([2, 4])
  filtro_escolaridade_baixa = df_sinan['CS_ESCOL_N'] <= 5
  # quantidades
  quantidade_viol_sexu = df_sinan[filtro_violencia_sexual].groupby(['VIOL_SEXU'])['VIOL_SEXU'].count().sum()
  quantidade_viol_sexu_homem = df_sinan[filtro_homem & filtro_violencia_sexual].groupby(['VIOL_SEXU'])['VIOL_SEXU'].count().sum()
  quantidade_viol_sexu_mulher = df_sinan[filtro_mulher & filtro_violencia_sexual].groupby(['VIOL_SEXU'])['VIOL_SEXU'].count().sum()
  quantidade_viol_domestica = df_sinan[filtro_viol_domestica].groupby(['LOCAL_OCOR'])['LOCAL_OCOR'].count().sum()
  quantidade_viol_domestica_homem = df_sinan[filtro_viol_domestica & filtro_homem].groupby(['LOCAL_OCOR'])['LOCAL_OCOR'].count().sum()
  quantidade_viol_domestica_mulher = df_sinan[filtro_viol_domestica & filtro_mulher].groupby(['LOCAL_OCOR'])['LOCAL_OCOR'].count().sum()
  quantidade_pardos_negros = len(df_sinan[filtro_negro_pardo])
  quantidade_baixa_escolaridade = len(df_sinan[filtro_escolaridade_baixa])

  # taxas
  quantidade = len(df_sinan)
  valor_int = int(valor)
  total_taxa = round(taxa(quantidade, valor_int),2)
  taxa_viol_sexu = round(taxa(quantidade_viol_sexu, valor_int),2)
  taxa_viol_homem = round(taxa(quantidade_viol_sexu_homem, valor_int),2)
  taxa_viol_mulher = round(taxa(quantidade_viol_sexu_mulher, valor_int),2)
  taxa_viol_domestica = round(taxa(quantidade_viol_domestica, valor_int),2)
  taxa_viol_domestica_homem = round(taxa(quantidade_viol_domestica_homem, valor_int),2)
  taxa_viol_domestica_mulher = round(taxa(quantidade_viol_domestica_mulher, valor_int),2)
  taxa_pardos_negros = round(taxa(quantidade_pardos_negros, valor_int), 2)
  taxa_escolaridade_baixa = round(taxa(quantidade_baixa_escolaridade, valor_int), 2)
  # print(df_sinan)
  print(f"quantidade: {quantidade}")
  print(f"Violencia sexual: {quantidade_viol_sexu}, quantidade homem: {quantidade_viol_sexu_homem}, quantiade mulher: {quantidade_viol_sexu_mulher}")
  print(f"Taxa viol sexu total: {taxa_viol_sexu}, homem: {taxa_viol_homem}, mulher: {taxa_viol_mulher}")
  print(f"Violencia domestica: {quantidade_viol_domestica}, quantidade homem: {quantidade_viol_domestica_homem}, quantidade mulher: {quantidade_viol_domestica_mulher}")
  print(f"Taxa Viol domest: {taxa_viol_domestica}, homem: {taxa_viol_domestica_homem}, mulher: {taxa_viol_domestica_mulher}")
  print(f"Quantidade pardos negros: {quantidade_pardos_negros}, Taxa: {taxa_pardos_negros}")
  print(f"Quantidade escolaridade baixa: {quantidade_baixa_escolaridade}, Taxa: {taxa_escolaridade_baixa}")

  print(f"Taxa: {total_taxa}, Ano: {ano}")

  df_novo = pd.DataFrame({'Sinan Ano':[ano],
                          'Total Viol':[quantidade],'Taxa': [total_taxa],
                          'Violencia Sexual':[quantidade_viol_sexu],'Taxa Violencia Sexual': [taxa_viol_sexu],
                          'Viol Homens':[quantidade_viol_sexu_homem],'Taxa Viol Homens': [taxa_viol_homem],
                          'Viol Mulheres':[quantidade_viol_sexu_mulher],'Taxa Viol Mulheres': [taxa_viol_mulher],
                          'Violencia Domestica':[quantidade_viol_domestica],'Taxa Violencia Domestica': [taxa_viol_domestica],
                          'Dom Homens':[quantidade_viol_domestica_homem],'Taxa Dom Homens': [taxa_viol_domestica_homem],
                          'Dom Mulheres':[quantidade_viol_domestica_mulher],'Taxa Dom Mulheres': [taxa_viol_domestica_mulher],
                          'Pardos e Negros':[quantidade_pardos_negros],'Taxa Pardos e Negros': [taxa_pardos_negros],
                          'Escolaridade Baixa':[quantidade_baixa_escolaridade],'Taxa Escolaridade Baixa': [taxa_escolaridade_baixa]
                          })
  df_estupro_viol = pd.concat([df_estupro_viol, df_novo], ignore_index=True)

/tmp/ipython-input-512322469.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinan['CS_RACA'] = pd.to_numeric(df_sinan['CS_RACA'], errors='coerce').fillna(0).astype(int)
/tmp/ipython-input-512322469.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinan['CS_ESCOL_N'] = pd.to_numeric(df_sinan['CS_RACA'], errors='coerce').fillna(0).astype(int)


quantidade: 2044
Violencia sexual: 832, quantidade homem: 117, quantiade mulher: 715
Taxa viol sexu total: 29.17, homem: 4.1, mulher: 25.07
Violencia domestica: 1123, quantidade homem: 312, quantidade mulher: 810
Taxa Viol domest: 39.37, homem: 10.94, mulher: 28.4
Quantidade pardos negros: 718, Taxa: 25.17
Quantidade escolaridade baixa: 1200, Taxa: 42.07
Taxa: 71.66, Ano: 2014


/tmp/ipython-input-512322469.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinan['CS_RACA'] = pd.to_numeric(df_sinan['CS_RACA'], errors='coerce').fillna(0).astype(int)
/tmp/ipython-input-512322469.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinan['CS_ESCOL_N'] = pd.to_numeric(df_sinan['CS_RACA'], errors='coerce').fillna(0).astype(int)


quantidade: 1936
Violencia sexual: 720, quantidade homem: 107, quantiade mulher: 613
Taxa viol sexu total: 24.7, homem: 3.67, mulher: 21.03
Violencia domestica: 1001, quantidade homem: 262, quantidade mulher: 739
Taxa Viol domest: 34.34, homem: 8.99, mulher: 25.35
Quantidade pardos negros: 548, Taxa: 18.8
Quantidade escolaridade baixa: 890, Taxa: 30.53
Taxa: 66.42, Ano: 2015


/tmp/ipython-input-512322469.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinan['CS_RACA'] = pd.to_numeric(df_sinan['CS_RACA'], errors='coerce').fillna(0).astype(int)
/tmp/ipython-input-512322469.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinan['CS_ESCOL_N'] = pd.to_numeric(df_sinan['CS_RACA'], errors='coerce').fillna(0).astype(int)


...

quantidade: 6951
Violencia sexual: 1271, quantidade homem: 122, quantiade mulher: 1149
Taxa viol sexu total: 41.08, homem: 3.94, mulher: 37.13
Violencia domestica: 5103, quantidade homem: 1135, quantidade mulher: 3966
Taxa Viol domest: 164.91, homem: 36.68, mulher: 128.17
Quantidade pardos negros: 3037, Taxa: 98.15
Quantidade escolaridade baixa: 4123, Taxa: 133.24
Taxa: 224.64, Ano: 2021


/tmp/ipython-input-512322469.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinan['CS_RACA'] = pd.to_numeric(df_sinan['CS_RACA'], errors='coerce').fillna(0).astype(int)
/tmp/ipython-input-512322469.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinan['CS_ESCOL_N'] = pd.to_numeric(df_sinan['CS_RACA'], errors='coerce').fillna(0).astype(int)


quantidade: 8831
Violencia sexual: 1618, quantidade homem: 175, quantiade mulher: 1443
Taxa viol sexu total: 54.24, homem: 5.87, mulher: 48.38
Violencia domestica: 6176, quantidade homem: 1471, quantidade mulher: 4705
Taxa Viol domest: 207.05, homem: 49.32, mulher: 157.74
Quantidade pardos negros: 4489, Taxa: 150.5
Quantidade escolaridade baixa: 5938, Taxa: 199.07
Taxa: 296.06, Ano: 2022


/tmp/ipython-input-512322469.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinan['CS_RACA'] = pd.to_numeric(df_sinan['CS_RACA'], errors='coerce').fillna(0).astype(int)
/tmp/ipython-input-512322469.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sinan['CS_ESCOL_N'] = pd.to_numeric(df_sinan['CS_RACA'], errors='coerce').fillna(0).astype(int)


quantidade: 11421
Violencia sexual: 1770, quantidade homem: 168, quantiade mulher: 1602
Taxa viol sexu total: 59.06, homem: 5.61, mulher: 53.46
Violencia domestica: 8383, quantidade homem: 1943, quantidade mulher: 6440
Taxa Viol domest: 279.72, homem: 64.83, mulher: 214.89
Quantidade pardos negros: 7357, Taxa: 245.49
Quantidade escolaridade baixa: 9522, Taxa: 317.73
Taxa: 381.09, Ano: 2023


In [138]:
df_estupro_viol.to_excel('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_filtrados/SINAN/estupros_viols_df.xlsx', index=False)

## Adicionando ao painel

In [139]:
df_painel_geral = pd.concat([df_painel_geral, df_estupro_viol], ignore_index=True)

# Notificações de violências interpessoais e autoprovocadas (dados SINAN)

In [ ]:
from pysus import SINAN
sinan = SINAN().load() # Loads the files from DATASUS

years = [i for i in range(2014,2026)]
sinan_files = sinan.get_files(dis_code=["VIOL"], year=years)

In [ ]:
sinan_parquets = sinan.download(sinan_files)

VIOLBR24.parquet: 100%|██████████| 1.90M/1.90M [02:26<00:00, 13.0kB/s]


## Salvar dados brutos

In [93]:
for parquet in sinan_parquets:
  df = pd.read_parquet(str(parquet))
  print()
  df.to_parquet('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_brutos/SINAN/'+str(parquet).split('/')[-1], engine='pyarrow', compression='snappy')

In [103]:
import pandas as pd
colunas_interesse = ['ID_AGRAVO', 'NU_ANO', 'SG_UF_NOT', 'CS_SEXO', 'CS_RACA', 'DT_NOTIFIC']
try:
  sinan_df_concat = pd.read_parquet('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_processados/sinan_concat.parquet', columns=colunas_interesse)
  print('Arquivo parquet carregado!')
except FileNotFoundError:
  sinan_df_concat = pd.DataFrame()


  frames = []
  for i in sinan_parquets:
    df_temporario = pd.read_parquet(str(i), columns=colunas_interesse)
    frames.append(df_temporario)

  sinan_df_concat = pd.concat(frames, ignore_index=True)

Arquivo parquet carregado!


In [95]:
print(sinan_df_concat.isna().sum().sum())

# sinan_df_concat.head()
sinan_df_concat['Ano'] = pd.to_datetime(sinan_df_concat['DT_NOTIFIC']).dt.year
sinan_df_concat['ID_AGRAVO'] = sinan_df_concat['ID_AGRAVO'].astype(str).str.upper().str.strip().str.replace('.', '', regex=False)

sinan_df_concat['CS_RACA'] = pd.to_numeric(sinan_df_concat['CS_RACA'], errors='coerce')
sinan_df_concat['CS_RACA'] = sinan_df_concat['CS_RACA'].fillna(9).astype(int)
sinan_df_filtrado = sinan_df_concat[sinan_df_concat['SG_UF_NOT'] == '53']

0


In [102]:
sinan_df_filtrado['CS_RACA'].value_counts()

,count
CS_RACA,
4,29604
9,20634
1,11018
2,4404
3,416
5,242


In [96]:
sinan_df_concat.to_parquet('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_processados/sinan_concat.parquet', compression='snappy')

In [140]:
regex_agressao = r'^(Y09|Y04|Y07|T74)'
regex_autoprovocada = r'^(X6\d|X7\d|X8[0-4]|T14)'


sinan_df_notificacao = pd.DataFrame()

for ano in range(2014,2026):
  ano_filtro = sinan_df_filtrado['Ano'] == ano
  agressao_filtro = sinan_df_filtrado['ID_AGRAVO'].str.contains(regex_agressao)
  autoprovocada_filtro = sinan_df_filtrado['ID_AGRAVO'].str.contains(regex_autoprovocada)
  homens_filtro = sinan_df_filtrado['CS_SEXO'] == 'M'
  mulheres_filtro = sinan_df_filtrado['CS_SEXO'] == 'F'
  ignorados_filtro = sinan_df_filtrado['CS_SEXO'] == 'I'
  branco_filtro = sinan_df_filtrado['CS_RACA'] == 1
  preta_filtro = sinan_df_filtrado['CS_RACA'] == 2
  amarela_filtro = sinan_df_filtrado['CS_RACA'] == 3
  indigena_filtro = sinan_df_filtrado['CS_RACA'] == 4




  quantidade_agressao = len(sinan_df_filtrado[ano_filtro & agressao_filtro])
  quantidade_autoprovocada = len(sinan_df_filtrado[ano_filtro & autoprovocada_filtro])
  quantidade_homens = len(sinan_df_filtrado[ano_filtro & homens_filtro])
  quantidade_mulheres = len(sinan_df_filtrado[ano_filtro & mulheres_filtro])
  quantidade_ignorados = len(sinan_df_filtrado[ano_filtro & ignorados_filtro])
  quantidade_branca = len(sinan_df_filtrado[ano_filtro & branco_filtro])
  quantidade_preta = len(sinan_df_filtrado[ano_filtro & preta_filtro])
  quantidade_amarela = len(sinan_df_filtrado[ano_filtro & amarela_filtro])
  quantidade_indigena = len(sinan_df_filtrado[ano_filtro & indigena_filtro])

  print(f"Ano: {ano}")
  print(f"Agressao: {quantidade_agressao}")
  print(f"Autoprovocada: {quantidade_autoprovocada}")
  print(f"Homens: {quantidade_homens}")
  print(f"Mulheres: {quantidade_mulheres}")
  print(f"Ignorados: {quantidade_ignorados}")
  print(f"Branca: {quantidade_branca}")
  print(f"Preta: {quantidade_preta}")
  print(f"Amarela: {quantidade_amarela}")
  print(f"Indígena: {quantidade_indigena}")


  print(f"\n\n\n")

  df_novo = pd.DataFrame(
      {
          "Sinan Ano":[ano],
          "Agressao":[quantidade_agressao],
          "Autoprovocada":[quantidade_autoprovocada],
          "Homens":[quantidade_homens],
          "Mulheres":[quantidade_mulheres],
          "Ignorados":[quantidade_ignorados],
          "Branca":[quantidade_branca],
          "Preta":[quantidade_preta],
          "Amarela":[quantidade_amarela],
          "Indígena":[quantidade_indigena]
      }
  )
  sinan_df_notificacao = pd.concat([sinan_df_notificacao, df_novo], ignore_index=True)


/tmp/ipython-input-4189260363.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  agressao_filtro = sinan_df_filtrado['ID_AGRAVO'].str.contains(regex_agressao)
/tmp/ipython-input-4189260363.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  autoprovocada_filtro = sinan_df_filtrado['ID_AGRAVO'].str.contains(regex_autoprovocada)


Ano: 2014
Agressao: 2044
Autoprovocada: 0
Homens: 526
Mulheres: 1517
Ignorados: 1
Branca: 361
Preta: 118
Amarela: 20
Indígena: 600




Ano: 2015
Agressao: 1936
Autoprovocada: 0
Homens: 518
Mulheres: 1418
Ignorados: 0
Branca: 294
Preta: 109
Amarela: 14
Indígena: 439




Ano: 2016
Agressao: 2383
Autoprovocada: 0
Homens: 590
Mulheres: 1793
Ignorados: 0
Branca: 339
Preta: 174
Amarela: 17
Indígena: 703




Ano: 2017
Agressao: 3534
Autoprovocada: 0
Homens: 790
Mulheres: 2744
Ignorados: 0
Branca: 661
Preta: 327
Amarela: 37
Indígena: 1197




Ano: 2018
Agressao: 4710
Autoprovocada: 0
Homens: 1068
Mulheres: 3641
Ignorados: 1
Branca: 832
Preta: 353
Amarela: 32
Indígena: 1614




Ano: 2019
Agressao: 6438
Autoprovocada: 0
Homens: 1544
Mulheres: 4894
Ignorados: 0
Branca: 1104
Preta: 414
Amarela: 35
Indígena: 2073




Ano: 2020
Agressao: 6220
Autoprovocada: 0
Homens: 1491
Mulheres: 4729
Ignorados: 0
Branca: 881
Preta: 392
Amarela: 26
Indígena: 2159




Ano: 2021
Agressao: 6951
Autoprovocada: 0
Homen

In [141]:
sinan_df_notificacao.to_excel('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_filtrados/SINAN/notificacoes_viols_agressao_autoprovocadas.xlsx', index=False)

## Adicionando ao painel geral

In [142]:
df_painel_geral = pd.concat([df_painel_geral, sinan_df_notificacao], ignore_index=True)

In [143]:
df_painel_geral.to_excel('/content/drive/MyDrive/Trabalho de Pesquisa Revisao/dados_filtrados/painel_integrado.xlsx', index=False)

In [ ]:
sinan_df_filtrado['ID_AGRAVO'].str.contains(regex_agressao).value_counts()

/tmp/ipython-input-3744472883.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sinan_df_filtrado['ID_AGRAVO'].str.contains(regex_agressao).value_counts()


,count
ID_AGRAVO,
True,66318


In [ ]:
sinan_df_filtrado['ID_AGRAVO'].str.contains(regex_autoprovocada).value_counts()

/tmp/ipython-input-3041520027.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  sinan_df_filtrado['ID_AGRAVO'].str.contains(regex_autoprovocada).value_counts()


,count
ID_AGRAVO,
False,66318


# Óbitos por causas externas e violência

In [ ]:
from pysus import SIM
import pandas as pd
sim = SIM().load() # Loads the files from DATASUS
years = [i for i in range(2014,2026)]
#
sim_files = sim.get_files("CID10", uf=["DF"], year=years)
#
sim_parquets = sim.download(sim_files)

KeyboardInterrupt: 